In [1]:
!pip install PyPDF2
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import PyPDF2

# ✅ Correct model name (don't install it via pip!)
model_name = "ibm-granite/granite-3.2-8b-instruct"

# ✅ Load tokenizer and model from Hugging Face (requires internet and optional login if gated)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# ✅ Set pad_token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 🧠 General response generator
def generate_response(prompt, max_length=1024):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.replace(prompt, "").strip()

# 📄 PDF text extractor
def extract_text_from_pdf(pdf_file):
    if pdf_file is None:
        return ""
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

# 1. 🌱 Eco Tips Generator
def eco_tips_generator(keywords):
    prompt = f"Generate practical and actionable eco-friendly tips for sustainable living related to: {keywords}."
    return generate_response(prompt, max_length=1000)

# 2. 📘 Policy Summarization
def policy_summarization(pdf_file, policy_text):
    content = extract_text_from_pdf(pdf_file) if pdf_file else policy_text
    prompt = f"Summarize this policy document. List key provisions, implications, and main points:\n\n{content}"
    return generate_response(prompt, max_length=1200)

# 3. ⚖️ Policy Comparison
def compare_policies(policy1, policy2):
    prompt = f"Compare these two policies. Highlight similarities, differences, and overall evaluation:\n\nPolicy 1:\n{policy1}\n\nPolicy 2:\n{policy2}"
    return generate_response(prompt, max_length=1200)

# 4. 🧠 Sustainability Quiz Generator
def generate_quiz(topic):
    prompt = f"Create a 5-question multiple-choice quiz on the topic: {topic}. Include 4 options per question and indicate the correct answers."
    return generate_response(prompt, max_length=1000)

# 🎛️ Gradio App Interface
with gr.Blocks() as app:
    gr.Markdown("# 🌍 Eco Assistant & 📄 Policy Analyzer")

    with gr.Tabs():
        # 🌱 Eco Tips
        with gr.TabItem("🌱 Eco Tips Generator"):
            keywords = gr.Textbox(label="Environmental Topic", placeholder="e.g., water conservation")
            tips_output = gr.Textbox(label="Eco Tips", lines=10)
            gen_button = gr.Button("Generate Tips")
            gen_button.click(eco_tips_generator, inputs=keywords, outputs=tips_output)

        # 📘 Policy Summarization
        with gr.TabItem("📘 Policy Summarizer"):
            pdf_input = gr.File(label="Upload Policy PDF", file_types=[".pdf"])
            policy_input = gr.Textbox(label="Or Paste Policy Text", lines=10)
            summary_output = gr.Textbox(label="Summary", lines=12)
            summarize_button = gr.Button("Summarize")
            summarize_button.click(policy_summarization, inputs=[pdf_input, policy_input], outputs=summary_output)

        # ⚖️ Policy Comparison
        with gr.TabItem("⚖️ Policy Comparison"):
            policy1 = gr.Textbox(label="Policy 1 Text", lines=8)
            policy2 = gr.Textbox(label="Policy 2 Text", lines=8)
            comparison_output = gr.Textbox(label="Comparison Result", lines=12)
            compare_button = gr.Button("Compare")
            compare_button.click(compare_policies, inputs=[policy1, policy2], outputs=comparison_output)

        # 🧠 Quiz Generator
        with gr.TabItem("🧠 Sustainability Quiz"):
            topic_input = gr.Textbox(label="Quiz Topic", placeholder="e.g., solar energy")
            quiz_output = gr.Textbox(label="Generated Quiz", lines=15)
            quiz_button = gr.Button("Generate Quiz")
            quiz_button.click(generate_quiz, inputs=topic_input, outputs=quiz_output)

# 🚀 Launch app
app.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a89ea34104d0f392ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
